In [116]:
import camb
from camb import model, initialpower
from matplotlib.collections import LineCollection
from matplotlib import colors as mcolors
from matplotlib import rc
import re
import matplotlib.pyplot as plt
from scipy.integrate import quad as qd
import numpy as np
import time as t
# from tqdm import tqdm, tqdm_pandas
# import pandas as pd
# tqdm.pandas()

In [117]:
pars = camb.CAMBparams()
pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0, tau=0.06)
pars.InitPower.set_params(As=2e-9, ns=0.965, r=0)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

def redshift_val(redshifts_values):
    pars.set_matter_power(redshifts=redshifts_values, kmax=2.0);

def PK(k, As, ns, alpha, beta):
    return As * (((k / 0.05) ** (ns - 1)) +  (alpha * (k / 0.05 )) + (beta * (((k / 0.05)**(-1))))) 

def prim_n(alpha, beta):
    pars.set_initial_power_function(PK, args=(1e-10, 0.96, alpha, beta));
    
def res():
    results = camb.get_results(pars)
    kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1, npoints = 200)
    return kh, z, pk

In [118]:
def b(z):
    return np.sqrt(1+z)

def w(z, w_0, w_1):
    return w_0 + w_1*(1/1+z)

def H(z):
    res = results.hubble_parameter(z)
    return res

def D(z):
    res = results.angular_diameter_distance(z)
    return res

# def Omega_m(z, omega_m_0, omega_lambda_0, omega_k_0, omega_r_0):
#     return (omega_m_0 * (1+z)**3) * (omega_lambda_0 + omega_k_0*(1+z)**2 + omega_m_0*(1+z)**3 + omega_r_0*(1+z)**4)**(-1)

def Omega_m(z):
    a = results.get_Omega('baryon', z)
    b = results.get_Omega('nu', z)
    c = results.get_Omega('cdm', z)
    d = results.get_Omega('neutrino', z)
    return a+b+c+d
    

def gamma(z):
    return gamma_0 + gamma_1 * (z / 1+z)

def f_g(z):
    return (Omega_m(z)**gamma(z)) * (1+etha)

def G(z):
    func_3 = lambda z_prime_3 : f_g(z_prime_3) * (1+z_prime_3)**(-1)
    integ_3 = qd(func_3, z, 0)[0]
    return np.exp(integ_3)
    
def betta(z):
    return f_g(z) / b(z)

def sigma_r(z):
    return sigma_z*c/H(z)

def sigma_v(z):
    return sigma_v_0/H(z)

def n_z(z):
    return (z**2) * np.exp(-(z/z_0)**3/2)

def r_z(z):
    func_4 = lambda z_prime_4 : H(z_prime_4)**(-1)
    integ_4 = qd(func_4, 0, z)[0]
    return integ_4*c

def vr_z(z):
    return (4*np.pi/3) * (f_sky) * ((r_z(z+0.05)**3) - (r_z(z-0.05)**3))

In [119]:
def P_k(z, alpha, beta):
    redshift_val([z])
    prim_n(alpha, beta)
    kh, z, pk = res()
    return kh, pk

def P_obs(z, mu, alpha, beta):
    pk_o = []
    kh, pk = P_k(z, alpha, beta)
    for i, j in enumerate(kh):
        pkval = (b(z)**2) * ((1 + betta(z) * mu**2)**2) * np.exp(-(j**2)*(mu**2)*((sigma_r(z)**2)+(sigma_v(z)**2))) * pk[0][i]
        pk_o.append(pkval)
    return kh, pk_o

def P_obs_val(k, z, mu, alpha, beta):
    _,_, P_value = P_obs(k, z, mu, alpha, beta)
    return P_value

def P_obs_z(kh, pk, z, mu):
    pk_o = []
    for i,j in enumerate(kh):
        pkval = (b(z)**2) * ((1 + betta(z) * mu**2)**2) * np.exp(-(j**2)*(mu**2)*((sigma_r(z)**2)+(sigma_v(z)**2))) * pk[0][i]
        pk_o.append(pkval)
    return kh, pk_o

In [120]:
H_0 = 67.5
gamma_0 = 0.545
gamma_1 = 0
etha = 0
sigma_z = 0.001
sigma_v_0 = 300
c = 2.99*100000
f_sky = 0.3636
del_z = 0.1
z_min = 0.65
z_max = 2.05
z_med = 0.9
z_0 = z_med/1.412
mu = np.arange(-1,1,0.01) 



In [121]:


def cov_sq(cov_matr):
    sq_cov = np.array([[0,0],[0,0]], dtype=np.float64)
    for i,p in enumerate(cov_matr):
        for j,q in enumerate(p):
            sq_cov[i][j] = abs(cov_matr[i][j])**(1/2)                       
    return sq_cov   

In [122]:
def pod(z, coff, alpha, beta):
    epsilon = 0.001
    kh1, pk1 = P_k(z, alpha, beta)
    deriv = []
    if coff == 'alpha':
        alpha += epsilon
        _, pk2 = P_k(z, alpha, beta)
        for i,j in enumerate(kh1):
            p1 = pk1[0][i]
            p2 = pk2[0][i]
            der = (p2 - p1)/epsilon
            deriv.append(der)
    elif coff == 'beta':
        beta += epsilon
        _, pk2 = P_k(z, alpha, beta)
        for i,j in enumerate(kh1):
            p1 = pk1[0][i]
            p2 = pk2[0][i]
            der = (p2 - p1)/epsilon
            deriv.append(der)
            
    return kh1, pk1, deriv

def val_finder(k, kh, lst):
    index = np.where(kh == k)[0][0]
    return lst[index]
    
def v(z, coff):
    alpha = 0.008
    beta = 0.002
    cof = (8*(np.pi)**2) ** (-1)
    v = vr_z(z)
    dk = 0.04628311744711677
    dmu = 0.05
    
    kh, pk1, derivA = pod(z, 'alpha', 0.008, 0.002)
    kh, pk1, derivB = pod(z, 'beta', 0.008, 0.002)
    
    if coff == "alpha" or coff == "beta":
        if coff == "alpha":
            c_lst = derivA
        else:
            c_lst = derivB
        mu_integ = []
        for i in mu:
            _, pk_o = P_obs_z(kh, pk1, z, i)
            func_var = list(map(lambda k: cof*v*((k**3)*(((n_z(z))**2)/((n_z(z)*val_finder(k,kh,pk_o)+1)**2))*(val_finder(k,kh,c_lst)**2)),kh))
            res = [i*dk for i in func_var]
            integ = np.sum(res)
            mu_integ.append(integ)
        mu_integ = [c*dmu for c in mu_integ]
        return np.sum(mu_integ)
    
    else:
        mu_integ = []
        for i in mu:
            _, pk_o = P_obs_z(kh, pk1, z, i)
            func_var = list(map(lambda k: cof*v*((k**3)*(((n_z(z))**2)/((n_z(z)*val_finder(k,kh,pk_o)+1)**2))*val_finder(k,kh,derivA)*val_finder(k,kh,derivB)),kh))
            res = [i*dk for i in func_var]
            integ = np.sum(res)
            mu_integ.append(integ)
        mu_integ = [c*dmu for c in mu_integ]
        return np.sum(mu_integ)
    
def fc_matr(z):
    t_start = t.time()
    a11 = v(z, 'alpha')
    a12 = v(z, 'alphabeta')
    a21 = a12
    a22 = v(z, 'beta')
    fish_matr = np.array([[a11,a12],[a21,a22]])
    cov_matr = np.linalg.inv(fish_matr)
    sq_cov = cov_sq(cov_matr)
    t_finish = t.time()
    print("for z : %s cov_matrix is \n"%str(z), sq_cov)
    print('Run Time :', t_finish-t_start,'Seconds')
    return sq_cov

In [ ]:
c_matr = fc_matr(0.65)